# HTML Text Extractor Utility

> Extracts unstructured text from scientific papers published as HTML files .

In [ ]:
#| default_exp utils.html_research_article_text_extractor

In [ ]:
#| export

import dataclasses

from typing import Dict, List, Union
from langchain_core.documents import Document
from langchain_community.document_loaders.base import BaseLoader
import re
from typing import Optional, List, Iterator, Mapping, Any, Dict
from dataclasses import field
import requests
import nltk
import trafilatura as traf
from bs4 import BeautifulSoup

In [ ]:
#| export

class TrafilaturaSectionLoader(BaseLoader):
    """Load `HTML` files and parse them with `trafilatura` into sections."""

    def __init__(self, file_path: str) -> None:
        """Initialize with a file path."""
        try:
            import trafilatura as traf
            from bs4 import BeautifulSoup
        except ImportError:
            raise ImportError(
                "trafilatura package not found, please install it with "
                "`pip install trafilatura`"
            )
        self.file_path = file_path
        self.bs_kwargs = {"features": "lxml"}

    def load(self) -> List[Document]:
        """Load HTML document into document objects."""

        docs = []        
        with open(self.file_path, encoding='utf-8') as f:
            j = traf.extract(f.read(), output_format='xml')
            soup = BeautifulSoup(j, "lxml-xml")
            offset = 0
            cs = soup.find('head', {'rend': 'h1'})

            dt = cs.get_text() + '\n'
            while cs.find_next_sibling() is not None:
                cs = cs.find_next_sibling()
                dt += cs.get_text() + '\n'
                if cs.name == 'head' and cs.get('rend') == 'h2':
                    md = {'c_ids': c_ids, 'e_id': e.id, 'e_type': e.type, 'i_type': 'HTMLFullText', 'offset': offset, 'length': len(dt)}
                    docs.append(Document(page_content=dt, metadata=md))
                    offset += len(dt)
                    dt = cs.get_text() + '\n'       

        return docs